# 本セクションの目次

1. メッセージキューとエコシステム
2. セクション２で構築した環境の説明
3. クラウド on メッセージキュー
4. メッセージキューがあると嬉しいこと 

# Apache Kafkaとそのエコシステム

![図1.2 Kafkaエコシステム](images/kafka_eco.png)

## Kafkaとは？
Apache Kafkaとは、分散メッセージキューです。  
メッセージキューとは、発生したストリーミングデータを保存しておける入れ物と取られてもOKです。

データはjsonでも問題ありませんし、単純な文字列でも問題ありません。  
また、今回は加えてストリーミング系のシステムでよく用いられるAvroについて紹介します。

## ストリーミングとは？
データを一行づつ、もしくはそれに近い単位で処理を行うことを指します。  
バッチ処理と単位が違うだけでやるべきことは同じです。

## パブリッシャー(プロデューサー)
データを生産する人。
データの生成方法は多様で、

- IoT(センサーデータ)
- Webブラウザのデータ(ログインした)
- コマンドライン

今回は、コマンドラインで基本を学びつつ最終的にはWebブラウザのデータをストリーミング処理してみたいと思います。

特にデータをkafkaに送信する行為のことを

## ブローカー
Kafkaにおけるデータの受付口。  
ブローカーによってどのトピックのどのパーティションにデータを格納するか？  
というのが全て決定される大事なところ。  

今回は1台のみですが、本番環境などでは複数台のブローカーを構築して冗長化します。

## Zookeeper
ブローカー同士のやり取りを取り持つ役割を持つ機能。  
分散処理では広く使われている。  
Zookeeperの登場なしに分散処理の発展はなかったというくらいすごい子。  

## トピック
データが格納される部分

コンシューマーの用途に合わせてトピックを作成することが多いです。
例えば、

- tsv形式で保存しておくトピック
- json形式で保存しておくトピック
- いろいろな情報を付与したデータを保存しておくトピック

といったような形です。

後で紹介するコンシューマーは、好きなトピックからデータを読み取って利用を行います。

## パーティション
簡単にいうと、トピックに対して冗長性を持たせたものがパーティション。  
パーティション2であればデータの冗長性は2となる。  

## コンシューマー(サブスクライバー)
データを消費する人、コマンドラインでデータをコンシュームすることも可能ですし、今回はPyspark(Spark Structured Streaming)でのデータを取得することをメインとしています

## シンク
コンシューマが受けたデータをどのように扱うかということ。  
一般にファイルに落とし込むことを「ファイルシンク」  
kakfaの別トピックに再度パブリッシュすることを「kafkaシンク」  
などと呼ぶ。

本コースでは

- ファイルシンク
- kafkaシンク
- メモリシンク
- コンソールシンク

いづれも行います。

# セクション2で構築した環境の説明
セクション2で作成を行なった環境は、前レクチャーで紹介したKafkaとエコシステムをdocker コンテナを使って表現したものになります。

![図1.2 環境構築](images/kafka_eco_env.png)

# クラウドonメッセージキュー

Apache kafkaはクラウド上で利用することもできますし、現在ではマネージドサービスも存在しています。  
また、Apache kafkaはメッセージキューと呼ばれる分類のもので、同様のサービスが各クラウドベンダーからリリースされています。

- Amazon Kinesiss
- Google pub/sub

オンプレの場合はApache Kafkaを自前で構築する必要がりますが、クラウドサービスを使えばほとんど手を煩わせることなく、メッセージキューを利用することが可能です。

今回はローカル環境でのコースになりますのでApache Kafkaを使いながらコースを行いますが、いずれのメッセージキューもメッセージキューなので本質は同じです。

# メッセージキューがあるとどんなことが嬉しいのか？
Kafkaがあると何が嬉しいのでしょうか？

- デカップリングが可能である

## デカップリングが可能である
デカップリングとは分離という意味です。
ここでは、システムのそれぞれの紐付きを分離するためにあるツールという意味合いです。

もし、メッセージキューがなければ(デカップリングされていなければ)、パブリッシャーからデータは直接コンシューマーに送信されることになりますが、この場合はコンシューマーのサービスが落ちてしまった場合データがロストする場合もあるのです。
また、仮にコンシューマーのシステムがエラーで停止した場合も影響は全くありません。  
コンシューマーが復旧したらそこからデータを取得すれば良いです。  

このような問題を解決すためにメッセージキューを利用します。
パブリッシャーとコンシューマーを分離するために、メッセージキューと言い換えてもいいかもしれません。

また、kafkaがあることで、ストリーミングデータの一時保存が可能になります。  
kafka自体はパーティションによって冗長化されていて、データの損失をできる限り防ぎます。